In [42]:
from openai import OpenAI 
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages=[]
    for message in messages:
        input_messages.append({"role": message["role"],"content":message["content"]})
    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000
    ).choices[0].message.content
    return response 

In [4]:
client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_CHATBOT_URL')
)
model_name = os.getenv("Model_Name")

# Get LLM response

In [5]:
messages=[{"role": "user", "content": "whats the capital of Germany?"}]
response = get_chatbot_response(client,model_name, messages)
response

'The capital of Germany is Berlin.'

# Prompt Engineering

## Structured Output

In [6]:
system_prompt="""
You are a helpful assistant that answers questions about capitals of countries.

Your Output should be in a structred JSON format exactly like the one below. You are not allowed to write anything other than the JSON object.
[
{
    "country": The country you will get the capital of,
    "capital": the capital of the country stated
}
]
"""

messages = [{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":"whats the capital of Germany?"})
response=get_chatbot_response(client,model_name,messages)
print(response)

[
{
  "country": "Germany",
  "capital": "Berlin"
}
]


In [7]:
type(response)

str

In [8]:
json_response = json.loads(response)
json_response

[{'country': 'Germany', 'capital': 'Berlin'}]

In [9]:
type(json_response)

list

## Input Structuring

In [10]:
user_input = """
get me the capitals of the following countries:
```
1. Italy
2. Germany
3. France
```
"""
messages = [{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":user_input})
response=get_chatbot_response(client,model_name,messages)
print(response)

[
{
    "country": "Italy",
    "capital": "Rome"
},
{
    "country": "Germany",
    "capital": "Berlin"
},
{
    "country": "France",
    "capital": "Paris"
}
]


In [11]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'Germany', 'capital': 'Berlin'},
 {'country': 'France', 'capital': 'Paris'}]

# Give the model Time to think (Chain of thought)

In [14]:
user_prompt = '''
Calculate the result of this equation: 1+3

your output should be in a structured JSON format exactly like the below. You are not allowed to write anything other than JSON object.
{
    result: The final number resulted from calculating the equation above.
}
'''
messages = [{"role":"user","content":user_prompt}]
response=get_chatbot_response(client,model_name,messages)
print(response)


{
    "result": 4
}


In [15]:
259/2*8654+91072*33-12971

4113098.0

In [19]:
user_prompt = '''
Calculate the result of this equation: 259/2*8654+91072*33-12971

your output should be in a structured JSON format exactly like the below. You are not allowed to write anything other than JSON object.
{
    result: The final number resulted from calculating the equation above.
}
'''
messages = [{"role":"user","content":user_prompt}]
response=get_chatbot_response(client,model_name,messages)
print(response)


{
    "result": 1444449.0
}


In [20]:
4113098.0-1444449.0


2668649.0

In [21]:
user_prompt = '''
Calculate the result of this equation: 259/2*8654+91072*33-12971

your output should be in a structured JSON format exactly like the below. You are not allowed to write anything other than JSON object.
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final result and feel free to write it in free text.
    result: The final number resulted from calculating the equation above.
}
'''
messages = [{"role":"user","content":user_prompt}]
response=get_chatbot_response(client,model_name,messages)
print(response)

{
    "steps": "First, we need to follow the BEDMAS order of operations. 
    1. Divide 259 by 2: 259/2 = 129.5
    2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121111
    3. Multiply 91072 by 33: 91072 * 33 = 3005016
    4. Add 1121111 and 3005016: 1121111 + 3005016 = 4126137
    5. Subtract 12971 from 4126137: 4126137 - 12971 = 4114026",
    "result": 4114026
}


# RAG - Retrival Augmented Generation

In [23]:
user_prompt = '''
whats new in Iphone 16?
'''
messages = [{"role":"user","content":user_prompt}]
response=get_chatbot_response(client,model_name,messages)
print(response)

Since Apple has not officially announced the iPhone 16, I'll provide you with some rumored features and speculations based on industry trends and leaks. Keep in mind that these are not confirmed and may change when the official announcement is made.

**Rumored Features:**

1. **Design:** The iPhone 16 is expected to have a similar design to the iPhone 15, with a stainless steel frame and a tougher glass back. However, some rumors suggest a new " pill-shaped" notch or a smaller notch.
2. **Display:** The iPhone 16 might feature a higher refresh rate (up to 120Hz) and a higher resolution (up to 6.7 inches) on the Pro models.
3. **Camera:** The iPhone 16 is expected to have improved camera capabilities, including a new periscope lens, a wider-angle lens, and enhanced low-light performance.
4. **Battery Life:** The iPhone 16 might have a larger battery, which could provide longer battery life.
5. **Charging:** The iPhone 16 could support faster charging speeds (up to 30W) and wireless char

In [28]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [26]:
user_prompt = f'''
{iphone_16}

whats new in Iphone 16?
'''
messages = [{"role":"user","content":user_prompt}]
response=get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger display sizes:
   - Base model: 6.1-inch
   - iPhone 16 Plus: 6.7-inch
   - iPhone 16 Pro: 6.3-inch
   - iPhone 16 Pro Max: 6.9-inch

2. Thinner bezels and a more durable Ceramic Shield.

3. New A18 chip (A18 Pro for Pro models) for improved performance, with:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

4. Upgraded camera systems:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button

5. Advanced audio features:
   - "Audio Mix" for refined audio capture during video recording

6. Extended battery life, especially in the iPhone 16 Pro Max.

7. Switch to USB-C for faster charging and data transfer.

8. Support for up to 2x faster video encoding in Pro models.

9. Starting prices remain consistent with previous generations:
   - iPhone 16: $799
 

# Automaricall extract context data from database

In [29]:
data = [iphone_16,samsung_s23]

In [30]:
user_prompt = """whats new in Iphone 16?"""

In [31]:
embedding_client = OpenAI(
    api_key=os.getenv('RUNPOD_Token'),
    base_url=os.getenv('RUNPOD_EMBEDDING_URL')
)

In [35]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input=text_input,model=model_name)
    embeddings=[]
    for embedding_object in  output.data:
        embeddings.append(embedding_object.embedding)
    return embeddings

In [48]:
user_prompt_embedding = get_embedding(embedding_client,model_name,user_prompt)[0]

In [49]:
data_embeddings = [get_embedding(embedding_client,model_name,text)[0] for text in data]

In [50]:
data_similarity = cosine_similarity([user_prompt_embedding],data_embeddings)

In [51]:
data_similarity

array([[0.74039999, 0.52683476]])

In [52]:
closest_entry_data_index=data_similarity.argmax()
closest_entry_data_index

0

In [53]:
data[closest_entry_data_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [54]:
user_prompt_with_data=f"""
{data[closest_entry_data_index]}
{user_prompt}
"""

In [55]:
messages = [{"role":"user","content":user_prompt_with_data}]
response=get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. **Larger displays**: 
   - Base model: 6.1-inch display
   - iPhone 16 Plus: 6.7-inch display
   - iPhone 16 Pro: 6.3-inch display
   - iPhone 16 Pro Max: 6.9-inch display

2. **Thinner bezels**: The iPhone 16 has thinner bezels than previous models.

3. **Durable Ceramic Shield**: The iPhone 16 features a more durable Ceramic Shield.

4. **New A18 chip (A18 Pro for Pro models)**: The iPhone 16 is powered by the new A18 chip, which offers significant performance improvements, including:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

5. **Upgraded camera systems**:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera
   - Enhanced by Apple's "Camera Control" button for more flexible photography options

6. **Advanced audio features**: The iPhone 16 features "Audio Mix," which uses machine le